In [1]:
import pandas as pd
import os
import numpy as np

In [76]:
data_file_name = "alexandras_outway_traj_merged_lane.csv"
data_path = "/home/data/data/pneuma/20181029_D8_1000_1030/"

reform_data_save_path = "/home/data/reform_data"
reform_data_name = "lanechange"

In [77]:
data.columns

Index(['track_id', 'speed', 'lon_acc', 'lat_acc', 'time', 'type', 'moving',
       'img_x', 'img_y', 'img_angle', 'coord_id', 'Lane'],
      dtype='object')

In [101]:
# read original data
data = pd.read_csv(data_path + data_file_name)

# columns to remain
col_li = ['track_id', 'speed', 'lon_acc', 'lat_acc', 'time', 'type', 
          'moving', 'img_x', 'img_y', 'img_angle', 'coord_id', 'Lane']

# leave columns that are needed
data = data[col_li]

In [102]:
data.head()

,track_id,speed,lon_acc,lat_acc,time,type,moving,img_x,img_y,img_angle,coord_id,Lane
0,40,2.8265,0.0166,-0.0668,0.0,Car,True,2424,1342,6.254072,597,3.0
1,40,2.6665,-0.2614,-0.1599,0.4,Car,True,2426,1343,6.268072,601,3.0
2,40,2.4949,-0.2269,-0.1515,0.8,Car,True,2428,1343,6.275755,615,3.0
3,40,2.0861,-0.3355,-0.1395,1.2,Car,True,2430,1342,6.279918,612,3.0
4,40,1.5341,-0.4492,0.0029,1.6,Car,True,2432,1342,6.282138,610,3.0


In [115]:
# if mean of Lane is not 0, then lane change occured
g = data.groupby(['track_id']).agg({'Lane': 'mean'})%1

lc = g.loc[g.Lane != 0]

# get list of track id with lane change occured
lc_true_track_id = lc.index.unique()

# get dataframe with lc true
lc_tracks = data[data['track_id'].isin(lc_true_track_id)]

# adding trigger column by groups (when lc occured)
lc_tracks['trigger'] = lc_tracks.groupby('track_id', group_keys=False).apply(lambda x: x['Lane'].diff().fillna(0))

# change trigger to boolean type
lc_tracks['trigger'] = lc_tracks['trigger'].astype('bool')

/tmp/ipykernel_5586/2812032484.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lc_tracks['trigger'] = lc_tracks.groupby('track_id', group_keys=False).apply(lambda x: x['Lane'].diff().fillna(0))
/tmp/ipykernel_5586/2812032484.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lc_tracks['trigger'] = lc_tracks['trigger'].astype('bool')


In [140]:
lc_tracks

,track_id,speed,lon_acc,lat_acc,time,type,moving,img_x,img_y,img_angle,coord_id,Lane,trigger
603,45,38.2121,0.0151,-0.0200,0.0,Car,True,2143,1375,0.071986,0,2.0,False
604,45,38.0674,-0.2361,-0.0508,0.4,Car,True,2172,1377,0.071486,10,2.0,False
605,45,37.9157,-0.2012,-0.0602,0.8,Car,True,2202,1379,0.069770,20,2.0,False
606,45,37.5489,-0.3050,-0.0863,1.2,Car,True,2231,1381,0.066833,30,1.0,True
607,45,37.0419,-0.4114,-0.1069,1.6,Car,True,2259,1382,0.063053,40,1.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
132694,2226,44.8746,-1.1174,-0.0321,862.4,Motorcycle,True,863,1342,0.007347,254,3.0,False
132695,2226,43.0067,-1.5647,-0.0102,862.8,Motorcycle,True,899,1342,0.006613,265,3.0,False
132696,2226,39.9298,-2.5947,0.0077,863.2,Motorcycle,True,933,1342,0.006666,275,3.0,False
132697,2226,36.4802,-1.9840,0.0137,863.6,Motorcycle,True,965,1342,0.007438,284,3.0,False


In [172]:
def data_generator(timestamp_period):
    import pickle
    import os

    folder_name = "timestamp_period_" + str(timestamp_period)
    os.makedirs(folder_name, exist_ok=True)

    path = os.path.join(os.path.abspath("."), folder_name)

    for id in lc_tracks.track_id.unique():
        count = 1 # for csv file name

        items = lc_tracks.loc[lc_tracks.track_id == id]

        idx = items.loc[items.trigger == True].index


        for i in idx:

            try:
                items.loc[i - timestamp_period]
                items.loc[i + timestamp_period]

            except Exception as e:
                continue

            else:
                filename = str(id) + "_lc" + '{0:02d}'.format(count) + '.csv'
                print(filename)

                parts = items.loc[i - timestamp_period: i + timestamp_period]

                # save in csv
                parts.to_csv(os.path.join(path, filename), mode='w')
                count+=1

In [173]:
data_generator(5)

51_lc01.csv
51_lc02.csv
71_lc01.csv
74_lc01.csv
165_lc01.csv
171_lc01.csv
207_lc01.csv
207_lc02.csv
207_lc03.csv
214_lc01.csv
218_lc01.csv
218_lc02.csv
218_lc03.csv
220_lc01.csv
221_lc01.csv
221_lc02.csv
223_lc01.csv
228_lc01.csv
232_lc01.csv
242_lc01.csv
246_lc01.csv
251_lc01.csv
258_lc01.csv
262_lc01.csv
262_lc02.csv
270_lc01.csv
279_lc01.csv
281_lc01.csv
288_lc01.csv
289_lc01.csv
295_lc01.csv
298_lc01.csv
299_lc01.csv
310_lc01.csv
311_lc01.csv
311_lc02.csv
311_lc03.csv
311_lc04.csv
317_lc01.csv
317_lc02.csv
317_lc03.csv
317_lc04.csv
321_lc01.csv
337_lc01.csv
345_lc01.csv
345_lc02.csv
357_lc01.csv
357_lc02.csv
357_lc03.csv
359_lc01.csv
359_lc02.csv
359_lc03.csv
360_lc01.csv
360_lc02.csv
376_lc01.csv
380_lc01.csv
381_lc01.csv
387_lc01.csv
387_lc02.csv
387_lc03.csv
398_lc01.csv
408_lc01.csv
424_lc01.csv
424_lc02.csv
444_lc01.csv
445_lc01.csv
449_lc01.csv
450_lc01.csv
451_lc01.csv
451_lc02.csv
452_lc01.csv
455_lc01.csv
455_lc02.csv
455_lc03.csv
458_lc01.csv
460_lc01.csv
467_lc01.csv
476

In [19]:
# get list of track_id with LaneChange
lc_id = list()

for i in range(data.shape[0]):
    item = data.iloc[i]
    # print(type(item['lc']))
    if item['lc'] == True:
        if item['track_id'] not in lc_id:     
            if data[data['track_id'] == item['track_id']].max()['time'] - data[data['track_id'] == item['track_id']].min()['time'] > 5:
                lc_id.append(item['track_id'])

data = data[data['track_id'].isin(lc_id)]

In [23]:
ndata = data.loc[data.type != 'Motorcycle']

In [141]:
lc_tracks.to_csv("reformed_data.csv", mode='w')